In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<version number>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,038 kB]
Hit:1

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-23 23:22:28--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.50MB/s    in 0.2s    

2022-11-23 23:22:28 (4.50 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://sierrabucket1117.s3.amazonaws.com/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz" 
spark.sparkContext.addFile(url)
health_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
health_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [5]:
#Count Rows
health_df.count()

5331449

In [6]:
#Print Schema
health_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
#Drop Null Review_ID, Product ID, Customer ID

dropna_df = health_df.na.drop(subset=["review_id","product_id","customer_id"]) 
dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [8]:
dropna_df.count()

5331449

In [9]:
# Load in a sql function to use columns
from pyspark.sql.functions import col


In [10]:
#Filter Unique review_ids


from pyspark.sql.functions import countDistinct
unique_review_df=dropna_df.select(countDistinct("review_id"))
unique_review_df.show()


+-------------------------+
|count(DISTINCT review_id)|
+-------------------------+
|                  5331449|
+-------------------------+



In [11]:
# Create user dataframe to match review_id_table table
health_review_id_df = dropna_df.select(["review_id", "customer_id", "product_id", "product_parent","review_date"])
health_review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R3EQSTM9PWRAL|     650634|B0091LBZSU|     578484426| 2015-08-31|
| RBWPRK17XKIXD|   19827510|B00PWW3LQ6|     456433146| 2015-08-31|
| RRSLOAF273XFC|    1520474|B00DKEWA92|     460764511| 2015-08-31|
|R3S8W9Q6SWIT8O|   23905905|B0015R3A7M|     135102038| 2015-08-31|
|R3QQ6NSLRVBFJC|   28215779|B006B8U8BG|     200666829| 2015-08-31|
|R2XYDBMHUVJCSX|   23260912|B00PFZFD8Y|     344168617| 2015-08-31|
|R1L94ESVC657A9|    7965616|B0034792JG|     291831570| 2015-08-31|
|R3DI4B8LDWFQ3K|    3443178|B000052XIA|     493678550| 2015-08-31|
|R1G7VV4HCXUQQL|   20838781|B004GCUK8A|       3902598| 2015-08-31|
|R3TTNYN1TAMUSU|   15267517|B00K7W1QAO|     633971374| 2015-08-31|
|R3AMX2D9HMWV6Z|   30105089|B0034KYA36|     854858899| 2015-08-31|
|R3PO3K321GY8HI|    1622207|B00C7UZTT8|     264608272| 2015-08

In [12]:
# Create user dataframe to match products table
health_products_df = dropna_df.select(["product_id", "product_title"])
health_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0091LBZSU|Demograss Capsule...|
|B00PWW3LQ6|Viva Labs #1 Prem...|
|B00DKEWA92|     Whip-it Cleaner|
|B0015R3A7M|Optimum Nutrition...|
|B006B8U8BG|Wet-Stop3 Bedwett...|
|B00PFZFD8Y|NatraCure Plantar...|
|B0034792JG|Glade Sense & Spr...|
|B000052XIA|Vagisil Anti-Itch...|
|B004GCUK8A|Glade Automatic S...|
|B00K7W1QAO|Kirkland Signatur...|
|B0034KYA36|Zyrtec Prescripti...|
|B00C7UZTT8|Pure Garcinia Cam...|
|B005F9XFN0|Now  Better Stevi...|
|B00Q8YSPTU|Now Foods Stevia ...|
|B00N34HBVG|Compression Thigh...|
|B002UN2UKS|Brownmed SEAL-TIG...|
|B003265KYK|Botanic Choice Gl...|
|B00939J5T6|Shibari SHIB-WHT ...|
|B00LJOQKRI|Eyekepper 5-pack ...|
|B002TG3QPO|Isopure Zero Carb...|
+----------+--------------------+
only showing top 20 rows



In [13]:
#Filter Unique Customer IDs

health_customer=dropna_df.select(countDistinct("customer_id"))
health_customer.show()

+---------------------------+
|count(DISTINCT customer_id)|
+---------------------------+
|                    3096336|
+---------------------------+



In [14]:
# Create user dataframe to match customers table
health_customers_df = dropna_df.groupBy("customer_id").count()
health_customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    4919528|    5|
|   51451778|    3|
|   12713799|    1|
|    8673341|    1|
|   42146698|    2|
|    1117644|    2|
|   28058398|    1|
|   14375645|    1|
|   24540309|    1|
|   39715602|    1|
|     654272|    1|
|    9015608|    5|
|   43920023|   42|
|   38209321|    1|
|   12328685|    1|
|    2866605|    2|
|   38273165|    5|
|   41066514|    1|
|   28377689|    2|
|   17018784|    2|
+-----------+-----+
only showing top 20 rows



In [15]:
# Rename Column for Customer Table 

health_customers_df.withColumnRenamed("count", "customer_count")


DataFrame[customer_id: int, customer_count: bigint]

In [16]:
# Create user dataframe to match vine table
health_vine_df = dropna_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
health_vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R3EQSTM9PWRAL|          3|            0|          0|   N|
| RBWPRK17XKIXD|          5|            0|          0|   N|
| RRSLOAF273XFC|          5|            1|          2|   N|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|
|R1L94ESVC657A9|          5|            0|          0|   N|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|
|R3AMX2D9HMWV6Z|          5|            0|          0|   N|
|R3PO3K321GY8HI|          1|            0|          0|   N|
| RVQ0UNYX6IEVK|          1|            0|          1|   N|
|R25E9NIGO2XQ3D|          5|            

Postgres Setup

In [17]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://"
config = {"user":"root", 
          "password": "", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write DataFrame to review_id table in RDS

health_review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to products table in RDS

health_products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write dataframe to customers  table in RDS

health_customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write dataframe to vine table in RDS

health_vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)